# Reference Code for Story Generator Project 

[Link](http://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/) 

In [1]:
# Small LSTM Network to Generate Text for Alice in Wonderland
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils


Using TensorFlow backend.


In [7]:
# load ascii text and covert to lowercase
# filename = "wonderland.txt"
filename = "E:/GitHub/StoryGeneration/Input Dataset/Spring-2016-cleaned.data"
raw_text = open(filename).read()
raw_text = raw_text.lower()
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)
raw_text = raw_text[0:10000]

Total Characters:  10474636
Total Vocab:  68


In [15]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    try:
        seq_in = raw_text[i:i + seq_length]
        seq_out = raw_text[i + seq_length]
        dataX.append([char_to_int[char] for char in seq_in])
        dataY.append(char_to_int[seq_out])
    except:
        pass
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

KeyboardInterrupt: 

In [14]:
print(X)

[[[ 0.67647059]
  [ 0.77941176]
  [ 0.52941176]
  ..., 
  [ 0.66176471]
  [ 0.75      ]
  [ 0.73529412]]

 [[ 0.77941176]
  [ 0.52941176]
  [ 0.72058824]
  ..., 
  [ 0.75      ]
  [ 0.73529412]
  [ 0.01470588]]

 [[ 0.52941176]
  [ 0.72058824]
  [ 0.79411765]
  ..., 
  [ 0.73529412]
  [ 0.01470588]
  [ 0.58823529]]

 ..., 
 [[ 0.76470588]
  [ 0.75      ]
  [ 0.57352941]
  ..., 
  [ 0.66176471]
  [ 0.01470588]
  [ 0.75      ]]

 [[ 0.75      ]
  [ 0.57352941]
  [ 0.19117647]
  ..., 
  [ 0.01470588]
  [ 0.75      ]
  [ 0.57352941]]

 [[ 0.57352941]
  [ 0.19117647]
  [ 0.73529412]
  ..., 
  [ 0.75      ]
  [ 0.57352941]
  [ 0.52941176]]]


In [ ]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)